In [1]:
#Import relevant modules.

import pygsti
import numpy as _np

from pygsti.algorithms import fiducialselection as FS

import matplotlib.pyplot as plt
%matplotlib inline

import time

In this notebook, we'll demonstrate how to select preparation and measurement fiducials for a standard two-qubit gate set.  By "standard", we mean that a) measurements are made in the computational (Z) basis (and state prep is |00>), and b) gate set consists of independent X pi/2 and Y pi/2 gates on each qubit.  Presumably there will be additional entangling gates available; however, we do not want (or need) such gates in our fiducial gate strings.  (Two-qubit operations will typically be of lower fidelity, so it is "safer" to use single-qubit operations for fiducials.)

In [2]:
#Build the gate set.  As mentioned above, no entangling operation is included; these results will be general for
#any two-qubit gate set that has access to the Gix, Giy, Gxi, and Gyi gates 
#(and prepares in the state |00> and performs measurements in the computational basis).
gs_target = pygsti.construction.build_gateset( [4], [('Q0','Q1')],['Gix','Giy','Gxi','Gyi'], 
                                            ["X(pi/2,Q1)", "Y(pi/2,Q1)", "X(pi/2,Q0)", "Y(pi/2,Q0)"], 
                                            prepLabels = ["rho0"], prepExpressions = ["0"], 
                                            effectLabels = ["E0","E1","E2"], effectExpressions = ["0","1","2"], 
                                            spamdefs={'upup': (0,0), 'updn': (0,1), 'dnup': (0,2), 'dndn': (0,-1) },
                                            basis="pp")

In [3]:
#Let's try to pick out a fiducial set.  

#First, we generate a candidate set which we'll attempt to prune.
#We could look at all gate strings of up to a fixed length (using pygsti.construction.list_all_gatestrings),
#but that grows quite rapidly.
#Instead, we'll look at the tensor product of the standard 1-qubit fiducial set with itself.
#This product set we define below.

#{} x 1q fid list
emptyList = pygsti.construction.gatestring_list([
        (),
        ('Gix',),
        ('Gix','Gix'),
        ('Gix','Gix','Gix'),
        ('Giy',),
        ('Giy','Giy','Giy')
    ])

#Gx x 1q fid list
XList = pygsti.construction.gatestring_list([
        ('Gxi',),
        ('Gxi','Gix',),
        ('Gxi','Gix','Gix'),
        ('Gxi','Gix','Gix','Gix'),
        ('Gxi','Giy',),
        ('Gxi','Giy','Giy','Giy')
    ])

#GxGx x 1q fid list
XXList = pygsti.construction.gatestring_list([
        ('Gxi','Gxi'),
        ('Gxi','Gxi','Gix',),
        ('Gxi','Gxi','Gix','Gix'),
        ('Gxi','Gxi','Gix','Gix','Gix'),
        ('Gxi','Gxi','Giy',),
        ('Gxi','Gxi','Giy','Giy','Giy')
    ])

#GxGxGx x 1q fid list
XXXList = pygsti.construction.gatestring_list([
        ('Gxi','Gxi','Gxi'),
        ('Gxi','Gxi','Gxi','Gix',),
        ('Gxi','Gxi','Gxi','Gix','Gix'),
        ('Gxi','Gxi','Gxi','Gix','Gix','Gix'),
        ('Gxi','Gxi','Gxi','Giy',),
        ('Gxi','Gxi','Gxi','Giy','Giy','Giy')
    ])

#Gy x 1q fid list
YList = pygsti.construction.gatestring_list([
        ('Gyi',),
        ('Gyi','Gix',),
        ('Gyi','Gix','Gix'),
        ('Gyi','Gix','Gix','Gix'),
        ('Gyi','Giy',),
        ('Gyi','Giy','Giy','Giy')
    ])

#Gy x 1q fid list
YYYList = pygsti.construction.gatestring_list([
        ('Gyi','Gyi'),
        ('Gyi','Gyi','Gyi','Gix',),
        ('Gyi','Gyi','Gyi','Gix','Gix'),
        ('Gyi','Gyi','Gyi','Gix','Gix','Gix'),
        ('Gyi','Gyi','Gyi','Giy',),
        ('Gyi','Gyi','Gyi','Giy','Giy','Giy')
    ])

testFidList = emptyList + XList + XXList + XXXList + YList + YYYList


In [4]:
#Don't worry if the optimize_integer_fiducials_slack function below throws a divide by zero warning;
#this just means one of the tested cases was *really* bad.

In [5]:
#We know that we should be able to find a prep fiducial set that has no more than 16 elements,
#so if we are finding sets that are larger than that, we can always increase slackFrac or fixedSlack
start = time.time()
prepFidList1_all = FS.optimize_integer_fiducials_slack(gs_target,testFidList,prepOrMeas='prep',initialWeights=None,
                                                       scoreFunc='all',slackFrac=.275)
end = time.time()
print('')
print("Fiducial selection completed in", end-start, "seconds.")
print("\n".join(map(str,sorted(prepFidList1_all,key=len))))

Complete initial fiducial set succeeds.
Now searching for best fiducial set.
  Starting fiducial set optimization. Lower score is better.
  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to bette

/home/enielse/research/pyGSTi/packages/pygsti/algorithms/fiducialselection.py:368: RuntimeWarning: divide by zero encountered in true_divide
  return sum(1./_np.abs(input_array))


In [6]:
#We know that we should be able to find a prep fiducial set that has no more than 16 elements,
#so if we are finding sets that are larger than that, we can always increase slackFrac or fixedSlack
start = time.time()
prepFidList1_worst = FS.optimize_integer_fiducials_slack(gs_target,testFidList,prepOrMeas='prep',initialWeights=None,
                                                         scoreFunc='worst',slackFrac=.275)
end = time.time()
print('')
print("Fiducial selection completed in", end-start, "seconds.")
print("\n".join(map(str,sorted(prepFidList1_worst,key=len))))

Complete initial fiducial set succeeds.
Now searching for best fiducial set.
  Starting fiducial set optimization. Lower score is better.
  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to bette

In [7]:
#We know that there might exist a fiducial measurement set with as few as 6 elements (as 6*3=18>16).
#However, repeated attempts to find one to date have failed.  We can reliably identify fiducial measurement sets
#with only 9 elements, so 9 should be considered an upper bound.  (If you do find a set with fewer than 9 elements,
#the pyGSTi team would love to hear from you!)
start = time.time()
measFidList1_all = FS.optimize_integer_fiducials_slack(gs_target,testFidList,prepOrMeas='meas',initialWeights=None,
                                                       scoreFunc='all',slackFrac=1)
end = time.time()
print('')
print("Fiducial selection completed in", end-start, "seconds.")
print("\n".join(map(str,sorted(measFidList1_all,key=len))))

Complete initial fiducial set succeeds.
Now searching for best fiducial set.
  Starting fiducial set optimization. Lower score is better.
  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to bette

/home/enielse/research/pyGSTi/packages/pygsti/algorithms/fiducialselection.py:368: RuntimeWarning: divide by zero encountered in true_divide
  return sum(1./_np.abs(input_array))


In [8]:
#Let's try the same as above, but with "worst" instead of "all" as the scoreFunc.
start = time.time()
measFidList1_worst = FS.optimize_integer_fiducials_slack(gs_target,testFidList,prepOrMeas='meas',initialWeights=None,
                                                       scoreFunc='worst',slackFrac=1)
end = time.time()
print('')
print("Fiducial selection completed in", end-start, "seconds.")
print("\n".join(map(str,sorted(measFidList1_worst,key=len))))

Complete initial fiducial set succeeds.
Now searching for best fiducial set.
  Starting fiducial set optimization. Lower score is better.
  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to better neighbor

  INVALID LEVEL:   Moving to bette

In [9]:
print("prep fid_all spectrum:\n", FS.test_fiducial_list(gs_target,prepFidList1_all,'prep',returnAll=True)[1])
print("prep fid_all 'all-score':", sum(FS.test_fiducial_list(gs_target,prepFidList1_all,'prep',
                                                             scoreFunc='all',returnAll=True)[2:]))
print("prep fid_all 'worst-score':", sum(FS.test_fiducial_list(gs_target,prepFidList1_all,'prep',
                                                             scoreFunc='worst',returnAll=True)[2:]))

prep fid_all spectrum:
 [ 0.04601333  0.08804562  0.11197634  0.18885363  0.19098301  0.27531678
  0.33869376  0.35895543  0.6341986   1.          1.14021158  1.30901699
  1.52707531  1.99774087  2.02250994  4.77040881]
prep fid_all 'all-score': 1088.0
prep fid_all 'worst-score': 347.725298007


In [10]:
print("prep fid_worst spectrum:\n", FS.test_fiducial_list(gs_target,prepFidList1_worst,'prep',returnAll=True)[1])
print("prep fid_worst 'all-score':", sum(FS.test_fiducial_list(gs_target,prepFidList1_worst,'prep',
                                                             scoreFunc='all',returnAll=True)[2:]))
print("prep fid_worst 'worst-score':", sum(FS.test_fiducial_list(gs_target,prepFidList1_worst,'prep',
                                                             scoreFunc='worst',returnAll=True)[2:]))

prep fid_worst spectrum:
 [ 0.08192728  0.11388855  0.13201528  0.19518522  0.27205661  0.29289322
  0.5         0.5         0.5         0.65555661  0.9189264   1.11499364
  1.70710678  2.18411013  2.19512844  4.63621184]
prep fid_worst 'all-score': 832.0
prep fid_worst 'worst-score': 195.295155964


In [11]:
#Interestingly, using the option "worst" instead of "all" yields a better scoring fiducial set, by both the "worst"
#and "all".

In [12]:
print("prep meas_all spectrum:\n", FS.test_fiducial_list(gs_target,measFidList1_all,'meas',returnAll=True)[1])
print("prep meas_all 'all-score':", sum(FS.test_fiducial_list(gs_target,measFidList1_all,'meas',
                                                             scoreFunc='all',returnAll=True)[2:]))
print("prep meas_all 'worst-score':", sum(FS.test_fiducial_list(gs_target,measFidList1_all,'meas',
                                                             scoreFunc='worst',returnAll=True)[2:]))

prep meas_all spectrum:
 [ 0.37146021  0.5         0.56368584  0.64534973  0.71248175  0.71922359
  0.74410254  0.75        0.75        1.91668037  2.          2.16101916
  2.52091388  2.77107311  2.78077641  7.0932334 ]
prep meas_all 'all-score': 158.065090329
prep meas_all 'worst-score': 24.2287055051


In [13]:
print("prep meas_worst spectrum:\n", FS.test_fiducial_list(gs_target,measFidList1_worst,'meas',returnAll=True)[1])
print("prep meas_worst 'all-score':", sum(FS.test_fiducial_list(gs_target,measFidList1_worst,'meas',
                                                             scoreFunc='all',returnAll=True)[2:]))
print("prep meas_worst 'worst-score':", sum(FS.test_fiducial_list(gs_target,measFidList1_worst,'meas',
                                                             scoreFunc='worst',returnAll=True)[2:]))

prep meas_worst spectrum:
 [ 0.37146021  0.5         0.56368584  0.64534973  0.71248175  0.71922359
  0.74410254  0.75        0.75        1.91668037  2.          2.16101916
  2.52091388  2.77107311  2.78077641  7.0932334 ]
prep meas_worst 'all-score': 158.065090329
prep meas_worst 'worst-score': 24.2287055051


In [14]:
for i in range(len(measFidList1_all)):
    print(sorted(measFidList1_all,key=len)[i], '\t', sorted(measFidList1_worst,key=len)[i], '\t', sorted(measFidList1_all,key=len)[i] == sorted(measFidList1_worst,key=len)[i])

{} 	 {} 	 True
GxiGxiGxiGiy 	 GxiGxiGxiGix 	 False
GyiGyiGyiGix 	 GyiGyiGyiGiy 	 False
GxiGxiGixGixGix 	 GxiGxiGixGixGix 	 True
GxiGxiGiyGiyGiy 	 GxiGxiGiyGiyGiy 	 True
GxiGxiGxiGixGix 	 GxiGxiGxiGixGix 	 True
GyiGyiGyiGixGix 	 GyiGyiGyiGixGix 	 True
GxiGxiGxiGixGixGix 	 GxiGxiGxiGiyGiyGiy 	 False
GyiGyiGyiGiyGiyGiy 	 GyiGyiGyiGixGixGix 	 False


In [15]:
#We have the same scores for "all" and "worst" for measurement fiducials, even though the fiducial sets themselves
#are not quite the same.

In [16]:
#Lastly, let's see if we can find a minimal set of measurement fiducials (size 6), using the same input set as before.

In [17]:
start = time.time()
measFidList1_all_force6 = FS.optimize_integer_fiducials_slack(gs_target,testFidList,prepOrMeas='meas',initialWeights=None,fixedNum=6,
                                                       scoreFunc='all',slackFrac=1)
end = time.time()
print('')
print("Fiducial selection completed in", end-start, "seconds.")
print("\n".join(map(str,sorted(measFidList1_worst,key=len))))

Complete initial fiducial set succeeds.
Now searching for best fiducial set.
  Starting fiducial set optimization. Lower score is better.
  Output set is required to be of size6
  Total number of fiducial sets to be checked is324632.0



/home/enielse/research/pyGSTi/packages/pygsti/algorithms/fiducialselection.py:414: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  scoreMx[:,colInd:colInd+int(numFids)] = fidArray[:,wtsLoc]
/home/enielse/research/pyGSTi/packages/pygsti/algorithms/fiducialselection.py:368: RuntimeWarning: divide by zero encountered in true_divide
  return sum(1./_np.abs(input_array))


  Switching!
  Switching!

Fiducial selection completed in 46.49062728881836 seconds.
{}
GxiGxiGxiGix
GyiGyiGyiGiy
GxiGxiGixGixGix
GxiGxiGiyGiyGiy
GxiGxiGxiGixGix
GyiGyiGyiGixGix
GxiGxiGxiGiyGiyGiy
GyiGyiGyiGixGixGix


In [18]:
FS.test_fiducial_list(gs_target,measFidList1_all_force6,'meas',scoreFunc='all',returnAll=True)

/home/enielse/research/pyGSTi/packages/pygsti/algorithms/fiducialselection.py:168: RuntimeWarning: divide by zero encountered in true_divide
  return sum(1./input_array)


(False, array([ -2.94335397e-16,  -1.41231424e-16,  -1.08737514e-16,
         -1.81977116e-17,   0.00000000e+00,   2.26097132e-16,
          2.77588083e-01,   4.34744748e-01,   5.00000000e-01,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.69387133e+00,   3.14892381e+00,   3.48662346e+00,
          5.45824857e+00]), inf)

In [19]:
#Sadly, this did not work!  However, one could try different input sets (or increasing fixedNum to 7 or 8, which would
#still be better than 9.)